# Cars4You - Handout Project | Group 42
## Notebook 1 — Models Lab
**Course:** MSc in Data Science and Advanced Analytics, NOVA IMS (2025/2026)  

**Author(s):** Group 42
- `Miguel Matos - 20221925`
- `Andre Nicolau - 20221918`
- `André Ferreira - 20250398`


---

#### <font> Table of Contents </font> <a class="anchor" id='toc'></a> 
1. [Imports](#Imports)  
----

# Imports
[Back to TOC](#toc)

In [ ]:
from utils.functions import *
from utils.Hermetic_Regressor import HermeticRegressor
import Preprocessing_pipelines.main_preprocessor_pipeline as main_preprocessor_pipeline


data = pd.read_csv("../data/data_cleaned.csv", index_col= "carID")
pd.set_option("display.max_columns", None)

In [2]:
data.head()

,Brand,model,car_age,price,transmission,mileage,fuelType,tax,mpg,engineSize,paintQuality%,previousOwners
carID,,,,,,,,,,,,
0,Audi,A3,8.0,10990,Manual,57097.0,Diesel,0.0,83.1,1.6,75.0,4.0
1,Audi,A7,7.0,22999,Semi-Auto,24337.0,Diesel,150.0,55.4,3.0,31.0,0.0
2,Audi,A4,8.0,14242,Manual,14900.0,Petrol,125.0,53.3,1.4,65.0,4.0
3,Audi,Q3,7.0,15998,Manual,37151.0,Petrol,145.0,51.4,1.4,55.0,1.0
4,Audi,A6,5.0,30850,Automatic,4553.0,Diesel,145.0,47.9,2.0,62.0,0.0


Let's drop "model" column as we don't intend to use it and to ease inital tests.

In [3]:
data.drop("model", axis = 1, inplace= True)

In [4]:
X = data.drop("price", axis= 1).select_dtypes(include=np.number)
y = data["price"]

In [20]:
#pipeline = Pipeline([ 
#           ("imputer", MissingValuesDealer()),
#           ("outliers", OutliersDealer()),
#           ("encoder", EncodingDealer()),
#           ("scaler", ScalingDealer()),
#           ("selector", FeatureSelectionDealer()),
#           ("model", RandomForestRegressor())      
#])

preprocessor_1 = main_preprocessor_pipeline.Preprocessor_Pipeline()
model_rf = RandomForestRegressor()
hermetic_regressor = HermeticRegressor(preprocessor_1, model_rf)

parameters = {
              "preprocessor__outlier_removal__threshold": [2,3]
}

mae_scorer = make_scorer(mean_absolute_error, greater_is_better=False)

random_search = RandomizedSearchCV(hermetic_regressor, parameters, scoring=mae_scorer, n_iter= 2, verbose=3, refit= "mae_scorer", n_jobs=-1)

In [21]:
random_search.fit(X, y)

Fitting 5 folds for each of 2 candidates, totalling 10 fits


[CV 2/5] END preprocessor__outlier_removal__threshold=2;, score=-2427.203 total time=  19.6s
[CV 1/5] END preprocessor__outlier_removal__threshold=3;, score=-4278.221 total time=  19.6s
[CV 1/5] END preprocessor__outlier_removal__threshold=2;, score=-4285.659 total time=  19.8s
[CV 3/5] END preprocessor__outlier_removal__threshold=2;, score=-3872.641 total time=  20.0s
[CV 4/5] END preprocessor__outlier_removal__threshold=2;, score=-2976.151 total time=  20.0s
[CV 5/5] END preprocessor__outlier_removal__threshold=2;, score=-2525.981 total time=  20.1s
[CV 2/5] END preprocessor__outlier_removal__threshold=3;, score=-2434.347 total time=  19.5s
[CV 3/5] END preprocessor__outlier_removal__threshold=3;, score=-3880.655 total time=  19.8s
[CV 4/5] END preprocessor__outlier_removal__threshold=3;, score=-2970.516 total time=  14.6s
[CV 5/5] END preprocessor__outlier_removal__threshold=3;, score=-2529.179 total time=  14.7s


RandomizedSearchCV(estimator=HermeticRegressor(model=RandomForestRegressor(),
                                               preprocessor=Preprocessor_Pipeline()),
                   n_iter=2, n_jobs=-1,
                   param_distributions={'preprocessor__outlier_removal__threshold': [2,
                                                                                     3]},
                   refit='mae_scorer',
                   scoring=make_scorer(mean_absolute_error, greater_is_better=False, response_method='predict'),
                   verbose=3)

It took 52 seconds to run.

Yayyyyyyy, já dá para correr istooooo.

In [22]:
pd.DataFrame(random_search.cv_results_)

,mean_fit_time,std_fit_time,mean_score_time,std_score_time,param_preprocessor__outlier_removal__threshold,params,split0_test_score,split1_test_score,split2_test_score,split3_test_score,split4_test_score,mean_test_score,std_test_score,rank_test_score
0,19.443057,0.182317,0.471578,0.019511,2,{'preprocessor__outlier_removal__threshold': 2},-4285.658889,-2427.203118,-3872.641158,-2976.150532,-2525.981414,-3217.527022,739.079160,1
1,17.267236,2.440253,0.369256,0.060990,3,{'preprocessor__outlier_removal__threshold': 3},-4278.221405,-2434.347315,-3880.655175,-2970.516431,-2529.178830,-3218.583831,736.615756,2


In [23]:
print("Best params:", random_search.best_params_)
print("Best score:", -random_search.best_score_)

Best params: {'preprocessor__outlier_removal__threshold': 2}
Best score: 3217.527022072648


In [24]:
best_model = random_search.best_estimator_

In [25]:
best_model.predict(X)


array([10420.5 , 23016.4 , 13509.63, ..., 19832.12, 16213.55, 18468.78])